# ML Options Pricing Model

### This notebook replicates the functionality of `main.py` while adding visualizations, statistical comparisons, and regression analysis.

In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from arch import arch_model
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [6]:
from data_loader import options_chain
from volatility_model import VolatilityModel
from pricing_model import PricingModel
from signal_generator import SignalGenerator

In [7]:
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

### Calculate historical volatility using GARCH

In [1]:
ticker = 'DVN'

In [ ]:
options, interest_rate, implied_volatility = options_chain(ticker)
stock_data = yf.download(ticker, period="5y")

In [ ]:
print("Stock Data:")
print(stock_data.tail())

In [ ]:
volatility_model = VolatilityModel()
stock_data['LogReturn'] = volatility_model.calculate_log_returns(stock_data['Adj Close'])
garch_volatility = volatility_model.calculate_garch_volatility(stock_data['Adj Close'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(stock_data.index, garch_volatility, label="GARCH Volatility", color="blue")
plt.title(f"Historical Volatility vs. GARCH Volatility for {ticker}")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.legend()
plt.show()

In [ ]:
print(f"Implied Volatility from Options Chain: {implied_volatility:.4f}")
print(f"Latest GARCH Volatility: {garch_volatility.iloc[-1]:.4f}")

In [ ]:
options['Moneyness'] = stock_data['Adj Close'].iloc[-1] / options['strikePrice']
options['TimeToExpiration'] = options['daysToExpiration'] / 365
options['RiskFreeRate'] = interest_rate / 100
options['HistoricalVolatility'] = garch_volatility.iloc[-1]

In [ ]:
options = volatility_model.interpolate_vol_surface(options)

### Train the XGBoost model

In [ ]:
pricing_model = PricingModel()
pricing_model.train_xgboost_model(options, interest_rate)

In [ ]:
feature_importance = pd.DataFrame({
    "Feature": options[['Moneyness', 'TimeToExpiration', 'RiskFreeRate', 'HistoricalVolatility']].columns,
    "Importance": pricing_model.model.feature_importances_
}).sort_values(by="Importance", ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=feature_importance, palette="viridis")
plt.title("Feature Importance in XGBoost Model")
plt.show()

### Predict option prices

In [ ]:
predicted_chain = pricing_model.predict_option_chain(
    stock_price=stock_data['Adj Close'].iloc[-1],
    options_chain=options,
    interest_rate=interest_rate,
    historical_volatility=garch_volatility.iloc[-1]
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(predicted_chain['mark'], predicted_chain['theoreticalOptionValue'], alpha=0.6)
plt.plot([min(predicted_chain['mark']), max(predicted_chain['mark'])], 
         [min(predicted_chain['mark']), max(predicted_chain['mark'])], 
         color="red", linestyle="--")
plt.xlabel("Market Price")
plt.ylabel("Predicted Price")
plt.title("Predicted vs. Market Prices")
plt.show()

### Regression Analysis

In [ ]:
X = predicted_chain[['mark']]
y = predicted_chain['theoreticalOptionValue']
regressor = LinearRegression()
regressor.fit(X, y)
y_pred = regressor.predict(X)

In [ ]:
print(f"R² Score: {r2_score(y, y_pred):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y, y_pred):.4f}")

### Trading Signals

In [ ]:
signal_generator = SignalGenerator(threshold=0.05)
signals = signal_generator.generate_trading_signals(predicted_chain)

In [ ]:
print("\nTrading Signals:")
print(signal_generator.format_output(signals))

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(signals['priceDifference'], bins=30, kde=True, color="purple")
plt.title("Distribution of Price Differences (Predicted vs. Market)")
plt.xlabel("Price Difference")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x="strikePrice", y="priceDifference", hue="signal", data=signals, palette="viridis")
plt.title("Trading Signals by Strike Price and Price Difference")
plt.xlabel("Strike Price")
plt.ylabel("Price Difference")
plt.legend(title="Signal")
plt.show()